In [7]:
import reconstruccion3D as r3D
import numpy as np
import matplotlib.pyplot as plot
import cv2
import pcl
#import sba

In [ ]:
%matplotlib inline

In [8]:
ruta = "img/monumento/"
K = np.array([[2759.48/4, 0, 1520.69/4, 0, 2764.16/4,1006.81/4, 0, 0, 1]]).reshape(3, 3)
d = np.array([0.0, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5)
a = r3D.Rescontruccion3D(ruta,K,d)

In [9]:
a.inicializacion()

#Con esta linea se le puede decir con que imagenes realizar la primera reconstruccion
#a.pts3D = a.construir_vista3D(0,1) 

In [10]:
a.salvar_nube("monumento.pcd")

<PointCloud of 393 points>

In [11]:
a.graficar_nube()

Incorporando Otras Vistas

In [12]:
vista_uno = a.vistas[0]
vista_dos = a.vistas[2]
vista_uno.buscar_feature(vista_dos,a.matcher)

In [13]:
#incorporando otras vistas.
new_feature = vista_uno.features[vista_dos]
old_feature = vista_uno.features[a.vistas[1]]

In [14]:
#Correspondiendo los puntos 2D con los 3D
match_utilizados =[]
j = 0
for i in range(a.FMask.shape[0]):
    if a.FMask[i]:
        punto3D = a.pts3D[j]
        match_utilizados.append((old_feature.matches[i],punto3D))
        j = j+1

In [15]:
#Determinando los features que se repiten en ambas imagenes.
match_c = []
puntos_3D = []
for old_match,punto3D in match_utilizados:
    for new_match in new_feature.matches:
        if new_match.queryIdx == old_match.queryIdx:
            match_c.append(old_match)
            puntos_3D.append(punto3D)

In [16]:
puntos_2D = []
for new_match in match_c:
    puntos_2D.append(new_feature.puntos_dos[new_match.trainIdx].pt)

In [17]:
retval,mask=cv2.findFundamentalMat(new_feature.match_puntos_uno,new_feature.match_puntos_dos,cv2.FM_RANSAC, 0.1, 0.99)

In [18]:
puntos_2D = np.array(puntos_2D)
puntos_3D = np.array(puntos_3D)
rvec,t,p = cv2.solvePnP(puntos_3D, puntos_2D, a.K,a.d)
R,j = cv2.Rodrigues(t,a.R)
P3 = np.hstack((R, a.t.reshape(3, 1)))
ptsH1,ptsH2,nuevo_matches = new_feature.homogeneizar_puntos(a.K,mask)
X = a.triangular(ptsH1,ptsH2,P3)

In [19]:
r = np.concatenate((X,a.pts3D))
r.shape
a.pts3D = r 
#aux = np.concatenate((aux,a.pts3D))

In [20]:
a.graficar_nube()